<a href="https://colab.research.google.com/github/Venkatakousik/FML/blob/main/nn_network_on_emnist_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
# Define the neural network architecture
import random
import numpy as np
import torch

# Set the random seed for reproducibility
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load the EMNIST dataset
train_dataset = datasets.EMNIST(root='./data', split='digits', train=True, download=True,
                                transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.EMNIST(root='./data', split='digits', train=False, download=True,
                               transform=transforms.Compose([transforms.ToTensor()]))

# Define the data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# en(train_loader)
# Initialize the neural network
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)

# Train the neural network
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward and optimizer(train_loader)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the learning rate
        scheduler.step()

        # Print the training progress
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs,loss.item()))
    


# Test the neural network
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_accuracy = 100 * correct / total
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch+1, num_epochs, loss.item(), test_accuracy))

Epoch [1/10], Loss: 0.0221
Epoch [2/10], Loss: 0.0226
Epoch [3/10], Loss: 0.0434
Epoch [4/10], Loss: 0.0114
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0027
Epoch [7/10], Loss: 0.0937
Epoch [8/10], Loss: 0.0347
Epoch [9/10], Loss: 0.0053
Epoch [10/10], Loss: 0.0003
Epoch [10/10], Loss: 0.0003, Test Accuracy: 98.61%
